In [ ]:
import tensorflow as tf
import random
import numpy
import cv2
import os

In [ ]:
dataset=list()
paths="C:\\Users\\KIIT\\Desktop\\Smart Attendance\\Dataset"

In [ ]:
def prepare_input(path,lab):
    for filename in os.listdir(path):
        img=cv2.imread(os.path.join(path,filename))
        img=cv2.resize(img,(100,100))
        dataset.append([img,lab])

In [ ]:
#train for anynumber of students
#i have used 4
prepare_input(paths+"\\0",0)
prepare_input(paths+"\\1",1)
prepare_input(paths+"\\2",2)
prepare_input(paths+"\\3",3)
prepare_input(paths+"\\4",4)

In [ ]:
print(len(dataset))

In [ ]:
random.shuffle(dataset)
for samples in dataset[:10]:
    print(samples[1])


In [ ]:
img=[]
lab=[]
for a,b in dataset:
    img.append(a)
    lab.append(b)

#we need to for a test train split
from sklearn.model_selection import train_test_split
x,x_valid,y,y_vaid=train_test_split(img,lab,test_size=0.25)

In [ ]:
import numpy as np
x=np.array(x).reshape(-1,100,100,3)
x_valid=np.array(x_valid).reshape(-1,100,100,3)
x=x.astype('float32')/255
x_valid=x_valid.astype('float32')/255

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard
NAME="Attendance"
tensorboard=TensorBoard(log_dir='logs/{}'.format(NAME))

In [ ]:
from keras import backend as K
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, TimeDistributed,Dropout, Activation, Flatten,Conv2D, MaxPooling2D,LSTM,Bidirectional
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
# create model
## training the CNN
cnn = Sequential()
#input
cnn.add(Conv2D(64, (3, 3), activation='relu', padding='same',input_shape=x.shape[1:]))
#1st layer
cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling2D((2, 2)))
cnn.add(Conv2D(128, (3, 3), activation='relu'))
cnn.add(MaxPooling2D((2, 2)))
cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling2D((2, 2)))
#converting to 1-d tensor
cnn.add(Flatten())

cnn.add(Dense(64,activation='relu'))
cnn.add(Dense(32,activation='relu'))
#model.add(Flatten())
cnn.add(Dense(5,activation='softmax'))
# Compile model
cnn.compile(loss='sparse_categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
# Fit the model
cnn.fit(x,y,epochs=25,validation_data=(x_valid,y_valid),batch_size=20,callbacks=[tensorboard])
cnn.save("prediction.h5")

In [ ]:
'''
#if you need to demo test the model
import tensorflow as tf
import numpy as np
import cv2
model=tf.keras.models.load_model("prediction.h5")
img=cv2.imread("C:\\Users\\KIIT\\Desktop\\Smart Attendance\\Dataset\\3\\xyz.jpg")
img=cv2.resize(img,(100,100))
test=np.array([img]).reshape(-1,100,100,3)
test=test.astype('float32')/255
x=model.predict(test)
print(x)
'''